In [1]:
# Wczytanie danych
import pandas as pd

df = pd.read_csv("Tweets.csv", sep=",", header=0)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [2]:
# Wyranie kolumn
df = df[["airline_sentiment","text"]]
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [3]:
# Normalizacja textu
import re
def norm1(row):
    return re.sub("[-()\"#/@;:<>{}`+=~|.!?,]",'',row["text"])
df["norm"] = df.apply(norm1, axis=1)
df.head()

,airline_sentiment,text,norm
0,neutral,@VirginAmerica What @dhepburn said.,VirginAmerica What dhepburn said
1,positive,@VirginAmerica plus you've added commercials t...,VirginAmerica plus you've added commercials to...
2,neutral,@VirginAmerica I didn't today... Must mean I n...,VirginAmerica I didn't today Must mean I need ...
3,negative,@VirginAmerica it's really aggressive to blast...,VirginAmerica it's really aggressive to blast ...
4,negative,@VirginAmerica and it's a really big bad thing...,VirginAmerica and it's a really big bad thing ...


In [4]:
# Zamiana wartości tekstowych (zmapowanie)

sentimental_map = {
    "positive":1,
    "neutral":0,
    "negative":-1
}
df["sentiment"] = df["airline_sentiment"].map(sentimental_map)
df.head()

,airline_sentiment,text,norm,sentiment
0,neutral,@VirginAmerica What @dhepburn said.,VirginAmerica What dhepburn said,0
1,positive,@VirginAmerica plus you've added commercials t...,VirginAmerica plus you've added commercials to...,1
2,neutral,@VirginAmerica I didn't today... Must mean I n...,VirginAmerica I didn't today Must mean I need ...,0
3,negative,@VirginAmerica it's really aggressive to blast...,VirginAmerica it's really aggressive to blast ...,-1
4,negative,@VirginAmerica and it's a really big bad thing...,VirginAmerica and it's a really big bad thing ...,-1


In [5]:
# Wektoryzacja
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
norm = df["norm"]
cv.fit(norm)
X = cv.transform(norm)
y = df[['sentiment']]

X.shape

(14640, 16529)

In [11]:
# Podział danych
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y)

num_classes = 3

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

X.shape

(14640, 16529)

In [7]:
# Tworzenie sieci neuronowej Keras
from keras import models
from keras.layers import Dense, Dropout

model = models.Sequential()
model.add(Dense(100, activation="relu", input_shape=(16529,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation="sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1653000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 1,663,403
Trainable params: 1,663,403
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Kompilacja modelu
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [9]:
# Trenowanie modelu

results = model.fit(
    X_train, y_train,
    epochs = 2,
    batch_size = 128,
    validation_data = (X_test, y_test)
)

Train on 10980 samples, validate on 3660 samples
Epoch 1/2
10980/10980 [==============================] - 6s 558us/step - loss: 0.4673 - acc: 0.7771 - val_loss: 0.3472 - val_acc: 0.8566
Epoch 2/2
10980/10980 [==============================] - 4s 379us/step - loss: 0.2764 - acc: 0.8891 - val_loss: 0.2952 - val_acc: 0.8815


In [10]:
# Sprawdzenie skuteczności 

score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  0.2951943359576939
Test accuracy:  0.8815118498489505
